In [13]:
import pandas as pd
import numpy as np
from pymoo.algorithms.unsga3 import UNSGA3
from pymoo.optimize import minimize
import pickle

from combined_problem import CombinedProblemDownSelect
from notebooks.optimization_problems.constraints import Requirements

In [14]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "telesat_post_processed.h5"

row_limit = -1

store_500km = pd.HDFStore(file_path)

instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)
instances_df = instances_500km_df

In [15]:
from notebooks.optimization_problems.design_vector import design_vector_default_scm, SystemParameters

sys_param = SystemParameters()
sys_param.margin_dB = 3.0
sys_param.B_Hz_array = np.array([0.1, 0.5, 1, 10, 50, 100, 200, 300]) * 1e6
sys_param.Gtx_dBi_bounds = (3., 30.)
sys_param.Ptx_dBm_bounds = (20., 43.)

modcods_df = pd.read_pickle('../dvbs2.pkl')

sys_param.EsN0_req_dB_array = modcods_df[['isend', 'isend_200MHz', 'isend_300MHz']].to_numpy()
sys_param.eta_bitsym_array = modcods_df[['eta', 'eta_200MHz', 'eta_300MHz']].to_numpy()
sys_param.eta_maee_array = modcods_df[['maee_12', 'maee_12_200MHz', 'maee_12_300MHz']].to_numpy()

requirements = Requirements()
requirements.min_throughput = 120e9
requirements.max_latency = 60.0 * 60.0 * 1.5

problem = CombinedProblemDownSelect(instances_df, sys_param, requirements=requirements, f_mask=np.array([0, 2, 3]))

sampling, crossover, mutation = design_vector_default_scm(problem.x_length, problem.x_indices)

algo_settings = {'pop_size': 100, 'n_offsprings': 25, 'n_max_gen': 5000}

settings = {}

# U-NSGA-III
from pymoo.algorithms.nsga3 import NSGA3
from pymoo.factory import get_reference_directions, get_visualization, get_termination

ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=32)
print("# dirs: %d" % len(ref_dirs))

#get_visualization("scatter").add(ref_dirs).show()
settings['UNSGA-III'] = {
    'label': 'unsga3',
    'algorithm': UNSGA3(
        pop_size=int(np.ceil(len(ref_dirs) / 100) * 100),
        n_offsprings=int(np.ceil(len(ref_dirs) / 100) * 100 / 4),
        sampling=sampling,#InitialSampling(),
        crossover=crossover,
        mutation=mutation,
        ref_dirs=ref_dirs,
        eliminate_duplicates=True,
    ),
    # 'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
    #                                                   n_last=30,
    #                                                   nth_gen=5,
    #                                                   n_max_gen=algo_settings['n_max_gen'],
    #                                                   n_max_evals=None)
    'termination': get_termination("time", "01:00:00")
}

setting = settings['UNSGA-III']

# dirs: 561


In [16]:
termination = setting['termination']
algorithm = setting['algorithm']

n_gen_inc = 5000

In [17]:
from os import path

file_name = 'Telesat_downselect_%s_%d_%d.pkl' % (setting['label'], algorithm.pop_size, algorithm.n_offsprings)

if path.exists(file_name):
    problem, setting, res = pickle.load(open(file_name, 'rb'))
    algorithm = res.algorithm
    algorithm.has_terminated = False
    n_gen = res.algorithm.n_gen
else:
    n_gen = 0

In [18]:
n_gen = n_gen + n_gen_inc
print(n_gen)

res = minimize(problem,
               algorithm,
               ('n_gen', n_gen),
               #termination=termination,
               seed=1,
               #save_history=True,
               verbose=True,
               copy_algorithm=False,
               )

print('Processes:', res.exec_time)
print("Best solution found: %s" % res.X)

pickle.dump((problem, setting, res), open(file_name, 'wb'))

5100
  101 |   15600 |  0.00000E+00 |  0.00000E+00 |     118 |  0.004505648 |            f
  102 |   15750 |  0.00000E+00 |  0.00000E+00 |     120 |  0.003631455 |            f
  103 |   15900 |  0.00000E+00 |  0.00000E+00 |     124 |  0.004652647 |            f
  104 |   16050 |  0.00000E+00 |  0.00000E+00 |     115 |  0.003794620 |            f
  105 |   16200 |  0.00000E+00 |  0.00000E+00 |     115 |  0.005737719 |            f
  106 |   16350 |  0.00000E+00 |  0.00000E+00 |     122 |  0.004849695 |            f
  107 |   16500 |  0.00000E+00 |  0.00000E+00 |     120 |  0.019791942 |        ideal
  108 |   16650 |  0.00000E+00 |  0.00000E+00 |     123 |  0.019407824 |        ideal
  109 |   16800 |  0.00000E+00 |  0.00000E+00 |     126 |  0.003813104 |            f
  110 |   16950 |  0.00000E+00 |  0.00000E+00 |     120 |  0.003952152 |            f
  111 |   17100 |  0.00000E+00 |  0.00000E+00 |     124 |  0.008596855 |            f
  112 |   17250 |  0.00000E+00 |  0.00000E+00 |  